In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from peft import PeftModel

BASE_MODEL = "TinyLlama/TinyLlama-1.1B-Chat-v1.0" # Основная модель
LORA_PATH = "/content/tinyllama-chemistry" # Веса LoRa после файн-тюнинга

# Загрущка модели
base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    device_map="auto",
    load_in_4bit=True,
    torch_dtype="auto"
)

model = PeftModel.from_pretrained(base_model, LORA_PATH) # Загрузка LoRa весов

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
# Токен для создания телеграм-бота
BOT_TOKEN = "7948169289:AAHW7sg7dD-zANU8slsBRS7-RbraXS4Q6J0"

In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
# === Обработка команды /start ===
async def start(update: Update, context: ContextTypes.DEFAULT_TYPE):
    welcome_message = (
        "👋 <b>Welcome to the <i>TinyLlama Chemistry Bot</i>!</b>\n\n"
        "Ask me anything about chemistry and I’ll do my best to help.\n\n"
        "🧪 <i>Example:</i> <code>What is a catalyst?</code>"
    )
    await update.message.reply_text(welcome_message, parse_mode="HTML")

# === Обработка сообщений ===
async def handle_message(update: Update, context: ContextTypes.DEFAULT_TYPE):
    user_input = update.message.text

    # "typing..." анимация
    await update.message.chat.send_action(action="typing")

    prompt = f"<s>[INST] {user_input} [/INST]"

    # Генерация
    result = pipe(prompt, max_new_tokens=200, do_sample=True, temperature=0.7)[0]['generated_text']

    # Удалим prompt из результата, если повторяется
    answer = result.replace(prompt, "").strip()[:3900]  # безопасная длина

    formatted = (
        f"<b>📥 Question:</b>\n<i>{user_input}</i>\n\n"
        f"<b>📤 Answer:</b>\n<pre>{answer}</pre>"
    )

    await update.message.reply_text(formatted, parse_mode="HTML")

# === Инициализация бота ===
app = ApplicationBuilder().token(BOT_TOKEN).build()
app.add_handler(CommandHandler("start", start))
app.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, handle_message))

# === Запуск бота ===
import asyncio

async def run_bot():
    print("🤖 Bot is running. Write to it in Telegram!")
    await app.run_polling()

await run_bot()